In [ ]:
# Trabalho 2 - Realce e Superresolução
# SCC0251 - Image Processing (01/2021)
# Fabiana Dalacqua Mendes - 9894536
# Pedro Henrique Nieuwenhoff - 10377729
import numpy as np
import random
import math

In [ ]:
def enhancement_superresolution():
    print()

In [ ]:
if __name__ == "__main__":
    enhancement_superresolution()